In [1]:
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [ ]:
llm=OpenAI(openai_api_key="****")
embedding = OpenAIEmbeddings(openai_api_key="***")

example of local llm

In [11]:
from langchain_community.llms import Ollama
llm1 = Ollama(model="llama3.1:latest")

Correct The Question

In [83]:

# Initialize the language model (llm)

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["question"],
    template="""correct cette:" {question}"
"""
)

# Initialize the LLM chain with the prompt
chain = LLMChain(llm=llm, prompt=prompt)




In [84]:
question = "ce qoit loi binomial"

# Run the chain with the question
answer12 = chain.run({"question": question})

# Output the answer
print(answer12)



Correcte : "la loi binomiale"


Build The RAG

In [3]:
from langchain_community.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA

In [4]:

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [6]:
loader = PyPDFLoader("cours Rappel probabilite.pdf")
pages = loader.load()

In [7]:
docs = text_splitter.split_documents(pages)

In [11]:
def extract_between_da_and_pdf(input_string):

    start = input_string.find("da/")
    end = input_string.find(".pdf")
    

    if start != -1 and end != -1:
      
        return input_string[start + len("da/"):end]
    else:
        return "Pattern not found"
def buildrag(full_path,persist_directory):
    
    loader = PyPDFLoader(full_path)
    pages = loader.load()
    for i in pages:
        i.metadata["topic"]=extract_between_da_and_pdf(i.metadata["source"])
        i.metadata["cours"]="exs"
        
    docs = text_splitter.split_documents(pages)
    vectorstoresen = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
    )        
            
    return 1

Example for the function

In [ ]:
full_path="test_pdfs/louka.pdf"
persist_directory = 'test/'
buildrag(full_path,persist_directory)

Build the vector-database

In [12]:
import os
pdf_directory = "da/"

persist_directory = 'final2/'
for filename in os.listdir(pdf_directory):
    # Check if the file is a PDF
    if filename.endswith(".pdf"):
        # Create the full path to the PDF file
        full_path = os.path.join(pdf_directory, filename)
        buildrag(full_path,persist_directory)
        

Chatbot Rag

In [13]:
persist_directory = 'final2/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [14]:
dd=vectordb.similarity_search("loi binomial")
dd

[Document(metadata={'cours': 'cours', 'page': 5, 'source': 'da/cours Rappel probabilite.pdf', 'topic': 'cours Rappel probabilite'}, page_content='3 Lois usuelles discr` etes\n3.1 La loi de Bernoulli B(p)\nLa variable al´ eatoire Xsuit une loi de Bernouilli de probabilit´ e p, siXvaut 1 ou\n0 avec les probabilit´ es respectives pet 1−p.D´ efinition 11 (Loi de Bernoulli B(p))\nOn note X∼ B(p) on a alors\n•P(X=x) =px(1−p)1−x,∀x∈ {0; 1}\n•E[X] =p\n•V[X] =p(1−p)\n3.2 La loi binomiale B(n, p)\nSi la variable al´ eatoire Xsuit une loi binomiale : X∼ B(n, p), cela veut dire que X\nest ´ egale au nombre de succ` es obtenus dans une s´ erie de n´ epreuves de Bernouilli\nind´ ependantes de probabilit´ e p.D´ efinition 12 (Loi binomiale B(n, p))\nOn note X∼ B(p) on a alors\n•P(X=k) =Ck\nnpk(1−p)n−k∀k∈ {0,1, . . . , n }\n•E[X] =np\n•V[X] =np(1−p)\n3.3 La loi de Poisson P(λ)\nSi une variable al´ eatoire Xrepr´ esente le nombre de succ` es dans un intervalle de\ntemps consid´ er´ e, alors Xob´ eit ` 

Determine the topic from the question

In [12]:
# Initialize the language model (llm)

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["question"],
    template="""
Vous recevrez une question en entrée, et votre tâche est de déterminer de quel sujet parmi les suivants elle relève :

Variables aléatoires continues
Tests d'hypothèses
Intervalle de confiance
Rappel probabilité
Estimation ponctuelle & distribution d’échantillonnage
Si la question ne correspond à aucun de ces sujets, veuillez simplement répondre : 'Je ne suis pas capable de déterminer le sujet.'

"""
)

# Initialize the LLM chain with the prompt
chain = LLMChain(llm=llm, prompt=prompt)



C:\Users\malek\AppData\Local\Temp\ipykernel_87792\3014908639.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm1, prompt=prompt)


In [8]:
question = "c'est qoit markov binomial"
answer1 = chain.run({"question": question})
print(answer1)
#Now, if the LLM determines the topic, you can search the database by filtering for that topic.

C:\Users\malek\AppData\Local\Temp\ipykernel_79048\1493202701.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer1 = chain.run({"question": question})


Rappel probabilité


#Now, if the LLM determines the topic, you can search the database by filtering for that topic.

chat with chatbot

In [15]:

persist_directory = 'final2/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [58]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Utilisez les informations suivantes pour répondre à la question à la fin. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, ne tentez pas d'inventer une réponse. Utilisez un maximum de trois phrases. Gardez la réponse aussi concise que possible. Terminez toujours par "merci d'avoir posé la question !".

{context}
Question : {question}
Réponse utile :
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [87]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 2},
                                 ),  # Set the number of retrieved documents to k
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [88]:
question = """
ce qoit loi binomial
"""
result = qa_chain({"query": question})
result["result"]

"La loi binomiale est une distribution de probabilité discrète qui modélise le nombre de succès dans une série d'épreuves indépendantes de même probabilité de succès. Elle est définie par deux paramètres : le nombre d'épreuves n et la probabilité de succès p. La variable aléatoire X suit alors une loi binomiale B(n, p), avec n étant le nombre de répétitions et p la probabilité de succès."

In [89]:
result

{'query': '\ndefinir loi binomial\n',
 'result': "La loi binomiale est une distribution de probabilité discrète qui modélise le nombre de succès dans une série d'épreuves indépendantes de même probabilité de succès. Elle est définie par deux paramètres : le nombre d'épreuves n et la probabilité de succès p. La variable aléatoire X suit alors une loi binomiale B(n, p), avec n étant le nombre de répétitions et p la probabilité de succès.",
 'source_documents': [Document(metadata={'page': 5, 'source': 'da/cours Rappel probabilite.pdf', 'topic': 'cours Rappel probabilite'}, page_content='3 Lois usuelles discr` etes\n3.1 La loi de Bernoulli B(p)\nLa variable al´ eatoire Xsuit une loi de Bernouilli de probabilit´ e p, siXvaut 1 ou\n0 avec les probabilit´ es respectives pet 1−p.D´ efinition 11 (Loi de Bernoulli B(p))\nOn note X∼ B(p) on a alors\n•P(X=x) =px(1−p)1−x,∀x∈ {0; 1}\n•E[X] =p\n•V[X] =p(1−p)\n3.2 La loi binomiale B(n, p)\nSi la variable al´ eatoire Xsuit une loi binomiale : X∼ B(n, p

chat with rag and activate filtre

In [97]:
prompt = PromptTemplate(
    input_variables=["question"],
    template="""Corrige les fautes d'orthographe dans le texte suivant sans changer le sens des phrases :
{question}
Texte corrigé :

"""
)

# Initialize the LLM chain with the prompt
chain1 =""
chain1 = LLMChain(llm=llm, prompt=prompt)

question = "dpnner moi definitionloi binomial"

# Run the chain with the question
answer1 = chain1.run({"question": question})

# Output the answer
print(answer1)

Donne-moi la définition de la loi binomiale.


In [16]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Utilisez les informations suivantes pour répondre à la question à la fin. Si vous ne connaissez pas la réponse, dites simplement que vous ne savez pas, ne tentez pas d'inventer une réponse. Utilisez un maximum de trois phrases. Gardez la réponse aussi concise que possible. Terminez toujours par "merci d'avoir posé la question !".

{context}
Question : {question}
Réponse utile :
"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
retriever = vectordb.as_retriever(search_kwargs={"k": 2, "filter": {"cours": "cours"}})
,  # Set the number of retrieved documents to k
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [22]:
question = """
Donne-moi la définition de la loi binomiale.
"""
result = qa_chain({"query": answer1})
result["result"]

" La loi binomiale est une loi de probabilité qui décrit le nombre de succès obtenus dans une série de n épreuves de Bernoulli indépendantes de probabilité p. Elle est notée X~B(n,p) et ses paramètres sont le nombre d'épreuves n et la probabilité de succès p. Merci d'avoir posé la question !"

In [23]:
result

{'query': '\nDonne-moi la définition de la loi binomiale.\n',
 'result': " La loi binomiale est une loi de probabilité qui décrit le nombre de succès obtenus dans une série de n épreuves de Bernoulli indépendantes de probabilité p. Elle est notée X~B(n,p) et ses paramètres sont le nombre d'épreuves n et la probabilité de succès p. Merci d'avoir posé la question !",
 'source_documents': [Document(metadata={'cours': 'cours', 'page': 5, 'source': 'da/cours Rappel probabilite.pdf', 'topic': 'cours Rappel probabilite'}, page_content='3 Lois usuelles discr` etes\n3.1 La loi de Bernoulli B(p)\nLa variable al´ eatoire Xsuit une loi de Bernouilli de probabilit´ e p, siXvaut 1 ou\n0 avec les probabilit´ es respectives pet 1−p.D´ efinition 11 (Loi de Bernoulli B(p))\nOn note X∼ B(p) on a alors\n•P(X=x) =px(1−p)1−x,∀x∈ {0; 1}\n•E[X] =p\n•V[X] =p(1−p)\n3.2 La loi binomiale B(n, p)\nSi la variable al´ eatoire Xsuit une loi binomiale : X∼ B(n, p), cela veut dire que X\nest ´ egale au nombre de succ` 